In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparing Training Data

In [2]:
rooms = []

for item in ['REAL','FAKE']:
 # Get all the file names
 all_rooms = os.path.join('/content/drive/MyDrive/DeepFake Detection/train_sample_videos/train_sample_videos/' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/DeepFake Detection/train_sample_videos/train_sample_videos/' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


NameError: name 'os' is not defined

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tag         110 non-null    object
 1   video_name  110 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
train_df.describe()

,tag,video_name
count,110,110
unique,2,46
top,REAL,/content/drive/MyDrive/Dataset/train_sample_vi...
freq,55,6


In [ ]:
train_df.tag.value_counts()

REAL    55
FAKE    55
Name: tag, dtype: int64

In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [ ]:
"""dataset_path = os.listdir('dataset/test')
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')"""

'dataset_path = os.listdir(\'dataset/test\')\nprint(dataset_path)\n\nroom_types = os.listdir(\'dataset/test\')\nprint("Types of activities found: ", len(dataset_path))\n\nrooms = []\n\nfor item in dataset_path:\n # Get all the file names\n all_rooms = os.listdir(\'dataset/test\' + \'/\' +item)\n\n # Add them to the list\n for room in all_rooms:\n    rooms.append((item, str(\'dataset/test\' + \'/\' +item) + \'/\' + room))\n    \n# Build a dataframe        \ntest_df = pd.DataFrame(data=rooms, columns=[\'tag\', \'video_name\'])\nprint(test_df.head())\nprint(test_df.tail())\n\ndf = test_df.loc[:,[\'video_name\',\'tag\']]\ndf\ndf.to_csv(\'test.csv\')'

In [ ]:
!pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-xce0txjf
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-xce0txjf
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=184397 sha256=290c393f9f3e4e49f2b23f97bcf2b3cdcb0a66bbfd658ab8616e18f72fba9564
  Stored in directory: /tmp/pip-ephem-wheel-cache-t_8pwjy1/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [ ]:
train_df = pd.read_csv("train.csv")
#test_df = pd.read_csv("test.csv")

#print(f"Total videos for training: {len(train_df)}")
#print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

,Unnamed: 0,video_name,tag
94,94,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
72,72,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
107,107,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
65,65,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
108,108,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
45,45,/content/drive/MyDrive/Dataset/train_sample_vi...,REAL
73,73,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
60,60,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
88,88,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
7,7,/content/drive/MyDrive/Dataset/train_sample_vi...,REAL


# Feed the videos to a network:


In [ ]:

IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 0s 0us/step


### Label Encoding
StringLookup layer encode the class labels as integers.

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['FAKE', 'REAL']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Finally, we can put all the pieces together to create our data processing utility.

In [ ]:
#print(train_data[0].shape)
#train_data[0]

In [ ]:
# no need to run

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 10
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()


    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()


    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "")
#test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")




Frame features in train set: (110, 10, 2048)
Frame masks in train set: (110, 10)
train_labels in train set: (110, 1)


In [ ]:

from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from google.colab import drive
from keras.layers import Activation, Dense,Dropout,GlobalAveragePooling2D
import gc

# ResNet50 Implementation

In [ ]:
input_tensor = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

base = ResNet50(weights="imagenet", include_top=False,input_tensor=input_tensor, input_shape=(IMG_SIZE, IMG_SIZE, 3))
#base.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

top = Dense(11, activation='softmax')(
            Dropout(0.5)(
                Dense(2048, activation='relu')(
                    Dropout(0.5)(
                        GlobalAveragePooling2D()(base.output)
                    )
                )
            )
        )

model = Model(input_tensor, top)

# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")


    x = keras.layers.LSTM(200, return_sequences=True)(
        frame_features_input, mask=mask_input
    )

    x = keras.layers.LSTM(200, return_sequences=True)(x)

    x = keras.layers.GRU(20)(x)
    #x = keras.layers.Dropout(0.4)(x)


    x = keras.layers.Dense(2048, activation="relu")(x)
    x = keras.layers.Dense(1024, activation="relu")(x)

    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)

    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)

    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True

In [ ]:
model.compile(keras.optimizers.Adam(lr=10e-5),loss = 'categorical_crossentropy',metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Model Training

In [ ]:
# Utility for running experiments.
def run_training():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    #seq_model = model
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([train_data[0], train_data[1]], train_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_training()

Epoch 1/100
3/3 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5325
Epoch 1: val_loss improved from inf to 0.69609, saving model to /tmp/video_classifier
3/3 [==============================] - 12s 2s/step - loss: 0.6929 - accuracy: 0.5325 - val_loss: 0.6961 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - ETA: 0s - loss: 0.6917 - accuracy: 0.7143
Epoch 2: val_loss did not improve from 0.69609
3/3 [==============================] - 0s 124ms/step - loss: 0.6917 - accuracy: 0.7143 - val_loss: 0.6988 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - ETA: 0s - loss: 0.6905 - accuracy: 0.7143
Epoch 3: val_loss did not improve from 0.69609
3/3 [==============================] - 0s 125ms/step - loss: 0.6905 - accuracy: 0.7143 - val_loss: 0.7015 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - ETA: 0s - loss: 0.6894 - accuracy: 0.7143
Epoch 4: val_loss did not improve from 0.69609
3/3 

# Model Saving

In [ ]:
sequence_model.save('/content/drive/MyDrive/models/ResNet50/ResNet50-deepfake-detection-tensor.h5')